In [9]:
import numpy as np
from numpy import linalg as LA
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from icecream import ic
from tqdm import trange
import time
from get_dataset_info import *

# %load_ext snakeviz
# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [10]:
# P1 = cv.get_canonical_camera()
# best_P2 = None
# best_X = None
# max_inliers = 0
# if epsilon > epsilon_E:

#     # P2_arr = cv.extract_P_from_E(E)
#     # X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm, x2_norm)
#     # P2_valid, X_valid = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr)
#     # valid_inliers = compute_valid_inliers(P1, P2_valid, X_valid, inliers)
#     # n_valid_inliers = np.sum(valid_inliers)
#     # print(n_valid_inliers, n_inliers)

#     # best_P2 = np.copy(P2_valid)
#     # best_X = np.copy(X_valid)
#     best_E = np.copy(E)
#     best_inliers = np.copy(inliers)
#     epsilon_E = epsilon
#     print('No. inliers:', n_inliers, end='\r')

#     # if n_valid_inliers > max_inliers:
#     #     best_inliers = np.copy(valid_inliers)
#     #     best_P2 = np.copy(P2_valid)
#     #     max_inliers = n_valid_inliers
#     #     print('No. valid inliers:', n_valid_inliers, end='\r')

#         # epsilon = max_inliers / n_points
#         # T = cv.compute_ransac_iterations(alpha, epsilon, n_samples)
#         # print('New T:', T, 'New epsilon:', epsilon)
#         # if t >= 4*T-1:
#         #     print('Bailout at iteration:', t, T)
#         #     break

# valid_inliers = compute_valid_inliers(P1, best_P2, best_X, best_inliers)
# n_valid_inliers = np.sum(valid_inliers)
# print('No. valid inliers:', n_valid_inliers, 'No. inliers:', np.sum(best_inliers))


In [11]:
def plot_3D_points(X):
    
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot(X[0], X[1], X[2], '.', ms=1, color='magenta', label='X')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.set_aspect('equal')
    ax.view_init(elev=-50, azim=-104, roll=20)
    ax.legend(loc="lower right")
    fig.tight_layout()
    plt.show()

In [12]:
def compute_E_validity(E):
    rank = LA.matrix_rank(E)
    valid = True if rank == 2 else False
    return valid

def compute_E_inliers(E, x1_norm, x2_norm, err_threshold):
    
    distance1_arr, distance2_arr = cv.compute_epipolar_errors(E, x1_norm, x2_norm)
    inliers = ((distance1_arr**2 + distance2_arr**2) / 2) < err_threshold**2
    n_inliers = np.sum(inliers)
    epsilon_E = n_inliers / x1_norm.shape[1]

    return epsilon_E, inliers

def verbose_E_robust(t, T_E, T_H, epsilon_E, epsilon_H, inliers, method):
    print('Iteration:', t, 'T_E:', T_E, 'T_H:', T_H, 'epsilon_E:', np.round(epsilon_E, 2), 'epsilon_H:', np.round(epsilon_H, 2), 'No. inliers:', np.sum(inliers), 'From:', method)

def compute_valid_inliers(P1, P2, X, inliers):

    x1_norm_valid = P1 @ X
    x2_norm_valid = P2 @ X
    valid_coords_P1 = x1_norm_valid[-1,:] > 0
    valid_coords_P2 = x2_norm_valid[-1,:] > 0
    valid_coords = valid_coords_P1 * valid_coords_P2
    valid_inliers = inliers * valid_coords

    return valid_inliers

def estimate_E_robust(K, x1_norm, x2_norm, min_its, max_its, scale_its, alpha, err_threshold_px, verbose=False):
    
    err_threshold = err_threshold_px / K[0,0]
    best_E = None
    best_inliers = None
    n_points = x1_norm.shape[1]
    n_E_samples = 8
    n_H_samples = 4
    best_epsilon_E = 0
    best_epsilon_H = 0
    T_E = max_its
    T_H = max_its

    t = 0
    while t < T_E and t < T_H:
        t += 1

        rand_mask = np.random.choice(n_points, n_E_samples, replace=False)
        E = cv.estimate_E_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], enforce=True, verbose=False)
        E_valid = compute_E_validity(E)

        if E_valid:
            epsilon_E, inliers = compute_E_inliers(E, x1_norm, x2_norm, err_threshold)
                
            if epsilon_E > best_epsilon_E:
                best_E = np.copy(E)
                best_inliers = np.copy(inliers)
                best_epsilon_E = epsilon_E
                T_E = cv.compute_ransac_iterations(alpha, best_epsilon_E, n_E_samples, min_its, max_its, scale_its)

                if verbose:
                    verbose_E_robust(t, T_E, T_H, best_epsilon_E, best_epsilon_H, best_inliers, method='E 8-point alg.')
        
        rand_mask = np.random.choice(n_points, n_H_samples, replace=False)
        H = cv.estimate_H_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], verbose=False)
        x2_norm_proj = cv.dehomogenize(H @ x1_norm)
        distance_arr = cv.compute_point_point_distance(x2_norm_proj, x2_norm)
        inliers = distance_arr**2 < err_threshold**2
        n_inliers = np.sum(inliers)
        epsilon_H = n_inliers / n_points

        if epsilon_H > best_epsilon_H:
            
            # num, Rs, Ts, Ns = cv2.decomposeHomographyMat(H, np.eye(3))
            R1, T1, R2, T2 = cv.homography_to_RT(H, x1_norm, x2_norm)
            E1 = cv.compute_E_from_R_and_T(R1, T1)
            E2 = cv.compute_E_from_R_and_T(R2, T2)

            E1_valid = compute_E_validity(E1)
            E2_valid = compute_E_validity(E2)

            if E1_valid:
                epsilon_E, inliers = compute_E_inliers(E1, x1_norm, x2_norm, err_threshold)
                    
                if epsilon_E > best_epsilon_E:
                    best_E = np.copy(E1)
                    best_inliers = np.copy(inliers)
                    best_epsilon_E = epsilon_E
                    best_epsilon_H = epsilon_H
                    T_E = cv.compute_ransac_iterations(alpha, best_epsilon_E, n_E_samples, min_its, max_its, scale_its)
                    T_H = cv.compute_ransac_iterations(alpha, best_epsilon_H, n_H_samples, min_its, max_its, scale_its)

                    if verbose:
                        verbose_E_robust(t, T_E, T_H, best_epsilon_E, best_epsilon_H, best_inliers, method='H 4-point alg.')

            if E2_valid:
                epsilon_E, inliers = compute_E_inliers(E2, x1_norm, x2_norm, err_threshold)
                    
                if epsilon_E > best_epsilon_E:
                    best_E = np.copy(E2)
                    best_inliers = np.copy(inliers)
                    best_epsilon_E = epsilon_E
                    best_epsilon_H = epsilon_H
                    T_E = cv.compute_ransac_iterations(alpha, best_epsilon_E, n_E_samples, min_its, max_its, scale_its)
                    T_H = cv.compute_ransac_iterations(alpha, best_epsilon_H, n_H_samples, min_its, max_its, scale_its)
                    
                    if verbose:
                        verbose_E_robust(t, T_E, T_H, best_epsilon_E, best_epsilon_H, best_inliers, method='H 4-point alg.')
    
    print('Bailout at iteration:', t)
    return best_E, best_inliers

In [13]:
def compute_sift_points(img1, img2, marg, verbose=False):
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < marg*n.distance:
            good_matches.append([m])

    draw_params = dict(matchColor=(255,0,255), singlePointColor=(0,255,0), matchesMask=None, flags=cv2.DrawMatchesFlags_DEFAULT)
    img_match = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches, None, **draw_params)

    x1 = np.stack([kp1[match[0].queryIdx].pt for match in good_matches],1)
    x2 = np.stack([kp2[match[0].trainIdx].pt for match in good_matches],1)
    x1 = cv.homogenize(x1, multi=True)
    x2 = cv.homogenize(x2, multi=True)

    des1 = np.stack([des1[match[0].queryIdx] for match in good_matches],0)
    des2 = np.stack([des2[match[0].trainIdx] for match in good_matches],0)

    if verbose:
        print('Number of matches:', np.size(matches,0))
        print('Number of good matches:', np.size(x1,1))

    return x1, x2, kp1, kp2, des1, des2, img_match

def compute_sift_points_TR(x1, des1, img2, marg, verbose=False):
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    sift = cv2.SIFT_create()
    kp2, des2 = sift.detectAndCompute(img2, None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < marg*n.distance:
            good_matches.append([m])

    x_idx = np.array([match[0].queryIdx for match in good_matches])
    x1 = np.stack([x1[:,match[0].queryIdx] for match in good_matches],1)
    x2 = np.stack([kp2[match[0].trainIdx].pt for match in good_matches],1)
    x2 = cv.homogenize(x2, multi=True)

    if verbose:
        print('Number of matches:', np.size(matches,0))
        print('Number of good matches:', np.size(x1,1))

    return x1, x2, x_idx

In [14]:
data_set = 1
K, img_names, init_pair, pixel_threshold = get_dataset_info(data_set)
K_inv = LA.inv(K)
imgs = cv.load_image(img_names, multi=True)
n_imgs = imgs.shape[0]
n_camera_pairs = n_imgs - 1
img1_init = imgs[init_pair[0]]
img2_init = imgs[init_pair[1]]

In [15]:
print('\n\n\n### Computing and saving SIFT-points ###\n')
print('## For rotation averaging ##')

sift_RA = False
marg = 0.7

if sift_RA:
    
    for i in range(n_camera_pairs):
        print("\nCamera pair:", i+1, "/", n_camera_pairs)
        img1 = imgs[i]
        img2 = imgs[i+1]
        x1, x2, _, _, _, _, _ = compute_sift_points(img1, img2, marg, verbose=True)
        np.save('data/dataset_{}_RA_x1_{}.npy'.format(data_set, i), x1)
        np.save('data/dataset_{}_RA_x2_{}.npy'.format(data_set, i), x2)



print('\n\n## For translation registration ##')

sift_TR = False
marg = 0.7

if sift_TR:
    
    print("\nImage:", init_pair[1]+1, "/", n_imgs, '(initial image pair)')
    x1_init, x2_init, kp1_init, kp2_init, des1_init, des2_init, _ = compute_sift_points(img1_init, img2_init, marg, verbose=True)
    np.save('data/dataset_{}_TR_x1_{}.npy'.format(data_set, init_pair[1]), x1_init)
    np.save('data/dataset_{}_TR_x2_{}.npy'.format(data_set, init_pair[1]), x2_init)

    X_visible = np.ones(x1_init.shape[1], dtype=bool)
    np.save('data/dataset_{}_TR_X_visible_{}.npy'.format(data_set, init_pair[0]), X_visible)
    np.save('data/dataset_{}_TR_X_visible_{}.npy'.format(data_set, init_pair[1]), X_visible)

    for i in range(n_imgs):

        if i != init_pair[0] and i != init_pair[1]:
            
            print("\nImage:", i+1, "/", n_imgs)
            img2 = imgs[i]
            x1, x2, x_idx = compute_sift_points_TR(x1_init, des1_init, img2, marg, verbose=True)
            x_idx = np.sort(x_idx)
            X_visible = []
            t = 0

            for j in range(x1_init.shape[1]):
                try:
                    if j == x_idx[t]:
                        X_visible.append(True)
                        t += 1
                    else:
                        X_visible.append(False)
                except:
                    X_visible.append(False)
            X_visible = np.array(X_visible)
                    
            np.save('data/dataset_{}_TR_X_visible_{}.npy'.format(data_set, i), X_visible)
            np.save('data/dataset_{}_TR_x1_{}.npy'.format(data_set, i), x1)
            np.save('data/dataset_{}_TR_x2_{}.npy'.format(data_set, i), x2)




### Computing and saving SIFT-points ###

## For rotation averaging ##


## For translation registration ##


In [16]:
print('\n\n\n### Loading SIFT-points ###\n')

# SIFT points for rotation averaging
x1s_norm_RA = []
x2s_norm_RA = []

for i in range(n_camera_pairs):

    x1 = np.load('data/dataset_{}_RA_x1_{}.npy'.format(data_set, i))
    x2 = np.load('data/dataset_{}_RA_x2_{}.npy'.format(data_set, i))
    x1_norm = cv.dehomogenize(K_inv @ x1)
    x2_norm = cv.dehomogenize(K_inv @ x2)
    x1s_norm_RA.append(x1_norm)
    x2s_norm_RA.append(x2_norm)
    
    
# SIFT points for translation registration
Xs_visible_TR = []
x1s_norm_TR = []
x2s_norm_TR = []

for i in range(n_imgs):

    if i != init_pair[0]:
        x1 = np.load('data/dataset_{}_TR_x1_{}.npy'.format(data_set, i))
        x2 = np.load('data/dataset_{}_TR_x2_{}.npy'.format(data_set, i))
        x1_norm = cv.dehomogenize(K_inv @ x1)
        x2_norm = cv.dehomogenize(K_inv @ x2)
        x1s_norm_TR.append(x1_norm)
        x2s_norm_TR.append(x2_norm)
    else:
        x1s_norm_TR.append(None)
        x2s_norm_TR.append(None)

    X_visible = np.load('data/dataset_{}_TR_X_visible_{}.npy'.format(data_set, i))
    Xs_visible_TR.append(X_visible)




### Loading SIFT-points ###



In [17]:
print('\n\n\n### Computing rotation averaging ###\n')

min_its = 0
max_its = 50000
scale_its = 1
alpha = 0.99
P1 = cv.get_canonical_camera()
abs_rots = [P1[:,:-1]]
abs_trans = [P1[:,-1]]

for i in range(n_camera_pairs):    

    print('\nCamera pair:', i+1, '/', n_camera_pairs)
    
    x1_norm = x1s_norm_RA[i]
    x2_norm = x2s_norm_RA[i]
    E, inliers = estimate_E_robust(K, x1_norm, x2_norm, min_its, max_its, scale_its, alpha, pixel_threshold, verbose=True)

    np.save('data/dataset_{}_RA_E_{}.npy'.format(data_set, i), E)
    np.save('data/dataset_{}_RA_inliers_{}.npy'.format(data_set, i), inliers)

    x1_norm_inliers = x1_norm[:,inliers]
    x2_norm_inliers = x2_norm[:,inliers]

    P2_arr = cv.extract_P_from_E(E)
    X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm_inliers, x2_norm_inliers)
    P2, _ = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr, verbose=True)

    R1 = abs_rots[i]
    R2 = P2[:,:-1]
    R2 = (R2 / LA.det(R2)) @ R1
    abs_rots.append(R2)

    T1 = abs_trans[i]
    T2 = P2[:,-1] + (R2 @ R1.T @ T1)
    abs_trans.append(T2)





### Computing rotation averaging ###


Camera pair: 1 / 8
Iteration: 2 T_E: 50000 T_H: 50000 epsilon_E: 0.05 epsilon_H: 0 No. inliers: 362 From: E 8-point alg.
Iteration: 18 T_E: 50000 T_H: 50000 epsilon_E: 0.13 epsilon_H: 0.0 No. inliers: 936 From: H 4-point alg.
Iteration: 19 T_E: 50000 T_H: 50000 epsilon_E: 0.15 epsilon_H: 0.0 No. inliers: 1026 From: E 8-point alg.
Iteration: 54 T_E: 50000 T_H: 50000 epsilon_E: 0.16 epsilon_H: 0.04 No. inliers: 1139 From: H 4-point alg.
Iteration: 593 T_E: 48.0 T_H: 50000 epsilon_E: 0.74 epsilon_H: 0.04 No. inliers: 5244 From: E 8-point alg.
Bailout at iteration: 593
No. valid coords for each camera pair: [ 5244  5244     0 10488]
Argmax(P2_arr): 3

Camera pair: 2 / 8
Iteration: 1 T_E: 50000 T_H: 50000 epsilon_E: 0.05 epsilon_H: 0 No. inliers: 380 From: E 8-point alg.
Iteration: 1 T_E: 50000 T_H: 50000 epsilon_E: 0.05 epsilon_H: 0.0 No. inliers: 406 From: H 4-point alg.
Iteration: 15 T_E: 50000 T_H: 50000 epsilon_E: 0.05 epsilon_H: 0.0 No. inlier

c:\Users\erikn\skola\EEN020-Computer-Vision\project\computer_vision.py:487: RuntimeWarning: divide by zero encountered in scalar divide
  T = scale * np.ceil(np.log(1-alpha) / np.log(1-epsilon**s))


Iteration: 65 T_E: 50000 T_H: 50000 epsilon_E: 0.18 epsilon_H: 0.1 No. inliers: 1378 From: H 4-point alg.
Iteration: 65 T_E: 50000 T_H: 50000 epsilon_E: 0.21 epsilon_H: 0.1 No. inliers: 1601 From: H 4-point alg.
Iteration: 117 T_E: 50000 T_H: 50000 epsilon_E: 0.21 epsilon_H: 0.1 No. inliers: 1607 From: E 8-point alg.
Iteration: 314 T_E: 2897.0 T_H: 50000 epsilon_E: 0.45 epsilon_H: 0.1 No. inliers: 3440 From: E 8-point alg.
Iteration: 701 T_E: 973.0 T_H: 50000 epsilon_E: 0.51 epsilon_H: 0.1 No. inliers: 3942 From: E 8-point alg.
Bailout at iteration: 973
No. valid coords for each camera pair: [3942 3942    0 7884]
Argmax(P2_arr): 3

Camera pair: 6 / 8
Iteration: 1 T_E: 50000 T_H: 50000 epsilon_E: 0.02 epsilon_H: 0 No. inliers: 119 From: E 8-point alg.
Iteration: 4 T_E: 50000 T_H: 50000 epsilon_E: 0.04 epsilon_H: 0 No. inliers: 292 From: E 8-point alg.
Iteration: 5 T_E: 50000 T_H: 50000 epsilon_E: 0.06 epsilon_H: 0 No. inliers: 490 From: E 8-point alg.
Iteration: 12 T_E: 50000 T_H: 50000

In [18]:
pair = 3
E = np.load('data/dataset_{}_RA_E_{}.npy'.format(data_set, pair))
inliers = np.load('data/dataset_{}_RA_inliers_{}.npy'.format(data_set, pair))
x1_norm = x1s_norm_RA[pair]
x2_norm = x2s_norm_RA[pair]

x1_norm_inliers = x1_norm
x2_norm_inliers = x2_norm

P1 = cv.get_canonical_camera()
P2_arr = cv.extract_P_from_E(E)
X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm_inliers, x2_norm_inliers)
P2, X = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr, verbose=True)

R1 = abs_rots[pair]
# X = R1.T @ X[:-1,:]
[print(abs_rots[i], LA.det(abs_rots[i])) for i in range(len(abs_rots))]

# plot_3D_points(X[:,inliers])

# Compute valid cheirality in E_robust
# Compute valid inliers in E_robust
# Fix rotation matrices 

No. valid coords for each camera pair: [ 7530  7530 15050    10]
Argmax(P2_arr): 2
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 1.0
[[ 0.02220614  0.00029968  0.00162995]
 [-0.00035413  0.02225301  0.00073324]
 [-0.001619   -0.00075713  0.02219606]] 1.1041745903649554e-05
[[ 2.80913925e-03  9.61083299e-05  3.66550790e-04]
 [-9.66690683e-05  2.83293328e-03 -1.94137274e-06]
 [-3.66403307e-04 -1.05767018e-05  2.81078217e-03]] 2.277547490025694e-08
[[ 1.36206624e-03  3.86131790e-05  1.52947889e-04]
 [-3.63437566e-05  1.37050666e-03 -2.23410397e-05]
 [-1.53502982e-04  1.81387184e-05  1.36243030e-03]] 2.577949260964069e-09
[[ 1.74209379e-05  5.73438093e-07  2.80351787e-06]
 [-5.54030084e-07  1.76449137e-05 -1.66413051e-07]
 [-2.80741774e-06  7.62324784e-08  1.74295788e-05]] 5.502479476044635e-15
[[ 1.03622787e-07  7.26647041e-09  2.46181785e-08]
 [-7.16673546e-09  1.06506155e-07 -1.27087884e-09]
 [-2.46473976e-08 -4.19091808e-10  1.03869478e-07]] 1.2166325384997335e-21
[[ 9.01100296e-12  6.67135716e

[None, None, None, None, None, None, None, None, None]

In [16]:
print('\n\n\n### Computing inliers between initial_pair[0] and all other images ###\n')

min_its = 1000
max_its = 20000
scale_its = 1
alpha = 0.99
compute_inliers = False

if compute_inliers:
    
    for i in range(n_imgs):

        if i != init_pair[0]:
            print('\nImage:', i+1, '/', n_imgs)

            # if i < init_pair[0]:
            #     x1_norm = x1s_norm_TR[i]
            #     x2_norm = x2s_norm_TR[i]
            # else:
            #     x1_norm = x2s_norm_TR[i]
            #     x2_norm = x1s_norm_TR[i]

            x1_norm = x1s_norm_TR[i]
            x2_norm = x2s_norm_TR[i]
            E, inliers = estimate_E_robust(K, x1_norm, x2_norm, min_its, max_its, scale_its, alpha, pixel_threshold, verbose=True)

            np.save('data/dataset_{}_TR_E_{}.npy'.format(data_set, i), E)
            np.save('data/dataset_{}_TR_inliers_{}.npy'.format(data_set, i), inliers)

            if i == init_pair[1]:
                # np.save('data/dataset_{}_TR_E_{}.npy'.format(data_set, init_pair[0]), E)
                np.save('data/dataset_{}_TR_inliers_{}.npy'.format(data_set, init_pair[0]), inliers)




### Computing inliers between initial_pair[0] and all other images ###


Image: 2 / 9
Iteration: 1 T_E: 20000 T_H: 20000 epsilon_E: 0.02 epsilon_H: 0 No. inliers: 27 From: E 8-point alg.
Iteration: 5 T_E: 20000 T_H: 20000 epsilon_E: 0.07 epsilon_H: 0.0 No. inliers: 86 From: H 4-point alg.
Iteration: 9 T_E: 20000 T_H: 20000 epsilon_E: 0.15 epsilon_H: 0.0 No. inliers: 190 From: E 8-point alg.
Iteration: 43 T_E: 20000 T_H: 20000 epsilon_E: 0.16 epsilon_H: 0.01 No. inliers: 211 From: H 4-point alg.
Iteration: 55 T_E: 20000 T_H: 20000 epsilon_E: 0.22 epsilon_H: 0.01 No. inliers: 288 From: E 8-point alg.
Iteration: 64 T_E: 20000 T_H: 20000 epsilon_E: 0.23 epsilon_H: 0.01 No. inliers: 298 From: E 8-point alg.
Iteration: 310 T_E: 1836.0 T_H: 20000 epsilon_E: 0.47 epsilon_H: 0.01 No. inliers: 613 From: E 8-point alg.
Iteration: 395 T_E: 1000 T_H: 20000 epsilon_E: 0.52 epsilon_H: 0.01 No. inliers: 675 From: E 8-point alg.
Bailout at iteration: 1000

Image: 3 / 9
Iteration: 1 T_E: 20000 T_H: 2

In [51]:
print('\n\n\n### Computing initial 3D-points ###\n')

pair = init_pair[1]
E = np.load('data/dataset_{}_TR_E_{}.npy'.format(data_set, pair))
inliers = np.load('data/dataset_{}_TR_inliers_{}.npy'.format(data_set, pair))
x1_norm = x1s_norm_TR[pair]
x2_norm = x2s_norm_TR[pair]

x1_norm_inliers = x1_norm
x2_norm_inliers = x2_norm

P1 = cv.get_canonical_camera()
P2_arr = cv.extract_P_from_E(E)
X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm_inliers, x2_norm_inliers)
P2, X = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr, verbose=True)

R1_init = abs_rots[init_pair[0]]
X = R1_init.T @ X[:-1,:]

plot_3D_points(X[:,inliers])




### Computing initial 3D-points ###

No. valid coords for each camera pair: [3512    0 1756 1756]
Argmax(P2_arr): 0


In [52]:
def estimate_T_robust(K, R, X, x_norm, min_its, max_its, scale_its, alpha, err_threshold_px, verbose=False):
    
    err_threshold = err_threshold_px / K[0,0]
    best_T = None
    best_inliers = None
    best_epsilon = 0
    n_points = x_norm.shape[1]
    n_samples = 2
    ransac_its = max_its

    t = 0
    while t < ransac_its:
        t += 1

        rand_mask = np.random.choice(n_points, n_samples, replace=False)
        
        # T1 = cv.estimate_T_DLT_1(x_norm[:,rand_mask], verbose=False)
        # T = T1

        T2 = cv.estimate_T_DLT_2(R, x_norm[:,rand_mask], verbose=False)
        T = T2
        # print(np.isclose(T1, T2), T1, T2)
        # time.sleep(2)

        x_norm_proj = cv.dehomogenize(R @ X + T[:,np.newaxis])
        distance_arr = cv.compute_point_point_distance(x_norm_proj, x_norm)
        inliers = distance_arr < err_threshold
        n_inliers = np.sum(inliers)
        epsilon = n_inliers / n_points

        if epsilon > best_epsilon:
            best_T = np.copy(T)
            best_inliers = np.copy(inliers)
            best_epsilon = epsilon
            ransac_its = cv.compute_ransac_iterations(alpha, best_epsilon, n_samples, min_its, max_its, scale_its)
            if verbose:
                print('Iteration:', t, 'T:', ransac_its, 'epsilon:', np.round(best_epsilon, 2), 'No. inliers:', np.sum(inliers))
    
    print('Bailout at iteration:', t)
    return best_T, best_inliers

In [53]:
print('\n\n\n### Computing translation registrations ###\n')

# Is the constraint img < init_pair[0] necessary?
# Save SIFT points for both x1 and x2 for TR
# Compute inliers for both x1 and x2 for TR

min_its = 0
max_its = 20000
scale_its = 1
alpha = 0.95
Ts = []

for i in range(n_imgs):

    print('\nImage:', i+1, '/', n_imgs)

    inliers = np.load('data/dataset_{}_TR_inliers_{}.npy'.format(data_set, i))
    X_visible = Xs_visible_TR[i]
    X_visible_inliers = X[:,X_visible][:,inliers]
    R = abs_rots[i]

    if i == init_pair[0]:
        x_norm = x1s_norm_TR[init_pair[1]]   
    else:
        x_norm = x2s_norm_TR[i]
        
    x_norm_inliers = x_norm[:,inliers] 
    
    # print(x_norm_inliers.shape, X_visible_inliers.shape, X_visible.shape, X.shape, X[:,X_visible].shape, inliers.shape)

    T = estimate_T_robust(K, R, X_visible_inliers, x_norm_inliers, min_its, max_its, scale_its, alpha, 5*pixel_threshold, verbose=True)
    Ts.append(T)




### Computing translation registrations ###


Image: 1 / 9
Iteration: 12 T: 20000 epsilon: 0.0 No. inliers: 1
Iteration: 197 T: 20000 epsilon: 0.0 No. inliers: 2


Iteration: 265 T: 20000 epsilon: 0.01 No. inliers: 3
Iteration: 546 T: 20000 epsilon: 0.01 No. inliers: 4
Iteration: 695 T: 20000 epsilon: 0.01 No. inliers: 5
Iteration: 999 T: 16214.0 epsilon: 0.01 No. inliers: 7
Iteration: 2382 T: 9808.0 epsilon: 0.02 No. inliers: 9
Bailout at iteration: 9808

Image: 2 / 9
Iteration: 23 T: 20000 epsilon: 0.0 No. inliers: 1
Iteration: 31 T: 20000 epsilon: 0.01 No. inliers: 6
Iteration: 866 T: 20000 epsilon: 0.01 No. inliers: 7
Iteration: 1130 T: 16850.0 epsilon: 0.01 No. inliers: 9
Iteration: 3913 T: 13648.0 epsilon: 0.01 No. inliers: 10
Iteration: 4867 T: 9478.0 epsilon: 0.02 No. inliers: 12
Bailout at iteration: 9478

Image: 3 / 9
Iteration: 60 T: 20000 epsilon: 0.01 No. inliers: 5
Iteration: 1306 T: 20000 epsilon: 0.01 No. inliers: 6
Iteration: 3847 T: 19751.0 epsilon: 0.01 No. inliers: 10
Iteration: 10683 T: 16323.0 epsilon: 0.01 No. inliers: 11
Bailout at iteration: 16323

Image: 4 / 9
Iteration: 3 T: 20000 epsilon: 0.0 No. inliers: 1
Iteration: 